In [ ]:
import os
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, Tool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.retrievers import WikipediaRetriever
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain import requests

llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-4o-mini-2024-07-18"
)


def save_file(docs, title):
    file = open(f"./{title}.txt", "w", encoding="utf-8")
    file.write(docs)


class SearchArgsSchema(BaseModel):
    term: str = Field(desciption="The term you will search for")


class WikipediaSearchTool(BaseTool):
    name="WikipediaSearchTool"
    description = """
    A tool getting results from Wikipedia by using the given keyword.
    """

    args_schema = Type[SearchArgsSchema] = SearchArgsSchema

    def wiki_search(topic):
        retriever = WikipediaRetriever(top_k_results=3)
        docs = retriever.get_relevant_documents(topic)
        return docs

    def _run(self, term):
        wiki_result = wiki_search(term)
        save_file(wiki_result,"wiki")
        return wiki_result   


class DDGSearchTool(BaseTool):
    name="DDGSearchTool"
    description = """
    A tool getting results from DuckDuckGo by using the given keyword.
    """

    args_schema = Type[SearchArgsSchema] = SearchArgsSchema

    def _run(self, term):
        ddg = DuckDuckGoSearchAPIWrapper()
        ddg_result =  ddg.run(term)
        save_file(ddg_result, "ddg")
        return ddg_result



agent = initialize_agent(
    llm=llm, 
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        WikipediaSearchTool(),
        DDGSearchTool(),
    ],
)

prompt = "Research about the XZ backdoor"

agent.invoke(prompt)